In [1]:
import qiskit
import numpy as np

from utils import LoadData
from QVC import VariationalClassifier
from qiskit.circuit.library import PauliFeatureMap

In [2]:
# Define quantum feature map
qfm = PauliFeatureMap(feature_dimension = 3,
                      reps = 2,
                      entanglement = 'linear',
                      alpha = 1,
                      paulis = ['Z', 'ZZ'])
print(qfm)

     ┌───┐┌──────────┐                                       ┌───┐»
q_0: ┤ H ├┤ U1(x[0]) ├──■─────────────────────────────────■──┤ H ├»
     ├───┤├──────────┤┌─┴─┐┌───────────────────────────┐┌─┴─┐└───┘»
q_1: ┤ H ├┤ U1(x[1]) ├┤ X ├┤ U1((π - x[0])*(π - x[1])) ├┤ X ├──■──»
     ├───┤├──────────┤└───┘└───────────────────────────┘└───┘┌─┴─┐»
q_2: ┤ H ├┤ U1(x[2]) ├───────────────────────────────────────┤ X ├»
     └───┘└──────────┘                                       └───┘»
«              ┌──────────┐                                   »
«q_0: ─────────┤ U1(x[0]) ├────────────────────────────────■──»
«              └──────────┘             ┌───┐┌──────────┐┌─┴─┐»
«q_1: ───────────────────────────────■──┤ H ├┤ U1(x[1]) ├┤ X ├»
«     ┌───────────────────────────┐┌─┴─┐├───┤├──────────┤└───┘»
«q_2: ┤ U1((π - x[1])*(π - x[2])) ├┤ X ├┤ H ├┤ U1(x[2]) ├─────»
«     └───────────────────────────┘└───┘└───┘└──────────┘     »
«                                                                           

In [3]:
# Define variational circuit
vc = qiskit.circuit.QuantumCircuit(3)
vcpv = qiskit.circuit.ParameterVector('theta', 16)
for i in range(2):
    vc.ry(vcpv[0+8*i] , 0)
    vc.ry(vcpv[1+8*i] , 1)
    vc.rz(vcpv[2+8*i] , 0)
    vc.rz(vcpv[3+8*i] , 1)
    vc.cz(0, 1)
    vc.ry(vcpv[4+8*i] , 1)
    vc.ry(vcpv[5+8*i] , 2)
    vc.rz(vcpv[6+8*i] , 1)
    vc.rz(vcpv[7+8*i] , 2)
    vc.cz(1, 2)
vc.draw()

┌──────────────┐┌──────────────┐   ┌──────────────┐┌───────────────┐   »
q_0: ┤ Ry(theta[0]) ├┤ Rz(theta[2]) ├─■─┤ Ry(theta[8]) ├┤ Rz(theta[10]) ├───»
     ├──────────────┤├──────────────┤ │ ├──────────────┤└┬──────────────┤   »
q_1: ┤ Ry(theta[1]) ├┤ Rz(theta[3]) ├─■─┤ Ry(theta[4]) ├─┤ Rz(theta[6]) ├─■─»
     ├──────────────┤├──────────────┤   └──────────────┘ └──────────────┘ │ »
q_2: ┤ Ry(theta[5]) ├┤ Rz(theta[7]) ├─────────────────────────────────────■─»
     └──────────────┘└──────────────┘                                       »
«                                                                            »
«q_0: ───────────────────────────────────■───────────────────────────────────»
«      ┌──────────────┐┌───────────────┐ │ ┌───────────────┐┌───────────────┐»
«q_1: ─┤ Ry(theta[9]) ├┤ Rz(theta[11]) ├─■─┤ Ry(theta[12]) ├┤ Rz(theta[14]) ├»
«     ┌┴──────────────┤├───────────────┤   └───────────────┘└───────────────┘»
«q_2: ┤ Ry(theta[13]) ├┤ Rz(theta[15]) ├─────────────────────────────────────»
«     └───────────────┘└───────────────┘                                     »
«        
«q_0: ───
«        
«q_1: ─■─
«      │ 
«q_2: ─■─
«

In [4]:
# Define boolean function
def bf(bs):
    if len([i for i in bs if i == '0']) > len([i for i in bs if i == '1']):
        return 1
    else:
        return -1

In [5]:
# Load Data
TrainData, TrainLabels, TestData, TestLabels = LoadData(0.2, 3, seed=30)

In [6]:
# Make classifier and train it
qvc = VariationalClassifier(qfm, vc, bf, False)
opt_param = np.load('./npy_files/Optimal_param_zzpfmc12_qiskit.npy')
qvc.optimal_params=opt_param

In [11]:
print(qvc.test(TestData, (-1)**TestLabels, verbose=True))

(0.6190476190476191, [0.1954, 0.1309, 0.899, 0.7103, 0.3968, 0.7149, 0.7698, 0.5595, 0.9376, 0.4131, 0.7911, 0.5107, 0.5215, 0.257, 0.3021, 0.8133, 0.6386, 0.6682, 0.3568, 0.6474, 0.3541], [False, False, True, True, False, True, True, True, True, False, True, True, True, False, False, True, True, True, False, True, False])


In [8]:
print(qvc.test(TrainData, (-1)**TrainLabels, verbose=True))

0.8271604938271605


In [9]:
len(TestData)

21

In [10]:
len(TrainData)

81